In [24]:
# Importing necessary packages
import numpy as np
import matplotlib.pyplot as plt
import operator
from mpl_toolkits.mplot3d import Axes3D
from sklearn import mixture
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans, AgglomerativeClustering, MeanShift, estimate_bandwidth
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.datasets import load_iris

In [19]:
# Function removes features with low variance
def rem_low_var(feat_mat,var_thresh):
    rem = VarianceThreshold(threshold=var_thresh)
    return rem.fit_transform(feat_mat)

In [20]:
# Function will run NMF on a given feature matrix and return a new matrix with desired dimensionality
def run_NMF(feat_mat,dim_out):
    model = NMF(n_components=dim_out,init='random', random_state=0)
    W = model.fit_transform(feat_mat)
    H = model.components_
    return W

In [2]:
# Function goal is to determine the right number of clusters for a given algorithm
# Runs algorithms with different numbers of clusters and calculates silhouette coefficient
# Returns the number of clusters that leads to the highest silhouette coefficient
def get_num_clust(feat_mat,alg):
    silhouettes = {}
    for k in range(2,11):
        if alg == 'agg':
            labels = clust_agg(feat_mat,k)
            avg_score = silhouette_score(feat_mat, labels)
            silhouettes[str(k)] = avg_score
        if alg == 'kmeans':
            (labels,centers) = clust_Kmeans(feat_mat,k)
            avg_score = silhouette_score(feat_mat, labels)
            silhouettes[str(k)] = avg_score
    num_clust = int(max(silhouettes.items(), key=operator.itemgetter(1))[0])
    return num_clust

In [3]:
# Function to estimate the bandwidth for MeanShift Clustering
def get_bw(feat_mat):
    bw = estimate_bandwidth(feat_mat)
    return bw

In [4]:
# Function runs MeanShift Clustering on a given feature matrix with a specified bandwidth parameters
# Returns the label for each sample
def clust_MS(feat_mat, bw):
    ms = MeanShift(bandwidth=bw).fit(feat_mat)
    labels = ms.labels_
    centers = ms.cluster_centers_
    return (labels, centers)

In [5]:
# Function runs K-means on a given feature matrix using a specified number of clusters
# Returns the feature matrix, the labels for each data point, and the cluster centers
def clust_Kmeans(feat_mat, num_clust):
    kmeans = KMeans(init='random', n_clusters=num_clust, n_init=100)
    kmeans.fit(feat_mat)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    return (labels, centers)

In [6]:
# Function runs Agglomerative Clustering on a given deature matrix using a specified number of clusters
# Retruns the label for each sample
def clust_agg(feat_mat, num_clust):
    labels = AgglomerativeClustering(n_clusters=num_clust).fit_predict(feat_mat)
    return labels

In [7]:
# Function calculates BIC for different numbers of clusters and different types of fits and returns resulting
# bic array
def get_bic(feat_mat):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 10)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            gmm.fit(fm)
            bic.append(gmm.bic(fm))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm
    bic = np.array(bic)
    return bic

In [8]:
# Function calculates BIC for different numbers of clusters and different types of fits and returns resulting
# aic array
def get_aic(feat_mat):
    lowest_aic = np.infty
    aic = []
    n_components_range = range(1, 10)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            gmm.fit(fm)
            aic.append(gmm.aic(fm))
            if bic[-1] < lowest_aic:
                lowest_aic = aic[-1]
                best_gmm = gmm
    aic = np.array(aic)
    return aic

In [25]:
# Reading in Vivek's feature matrix
fm = np.genfromtxt('hbn_vertexstats.csv', delimiter=',')
fm = fm[1:,1:]

#Reducing dimensionality
fm = run_NMF(fm, int(fm.shape[1]/3))

# Getting true labels
true_labels = np.zeros(91)
for i in range(0, fm.shape[0]):
    true_labels[i] = fm[i,0]

# Agglomerative Clustering & Silhouette Score
numc_agg = get_num_clust(fm, 'agg')
labels_agg = clust_agg(fm, numc_agg)
ari_agg = adjusted_rand_score(true_labels, labels_agg)

# Kmeans & Silhouette Score
numc_kmeans = get_num_clust(fm, 'kmeans')
(labels_kmeans, centers_kmeans) = clust_Kmeans(fm, numc_kmeans)
ari_kmeans = adjusted_rand_score(true_labels, labels_kmeans)

# MeanShift Clustering & Estimate Bandwidth
bw = get_bw(fm)
(labels_ms, centers_ms) = clust_MS(fm, bw)
ari_ms = adjusted_rand_score(true_labels, labels_ms)

# GMM and BIC
bic = get_bic(fm)
low_bc = np.argmin(bic)

# GMM and AIC
aic = get_aic(fm)
low_aic = np.argmin(aic)

# Getting BIC and AIC labels
gmm = mixture.GaussianMixture(n_components=2, covariance_type='full')
labels_bic = gmm.fit_predict(fm)
ari_bic = adjusted_rand_score(true_labels, labels_bic)
gmm = mixture.GaussianMixture(n_components=6, covariance_type='full')
labels_aic = gmm.fit_predict(fm)
ari_aic = adjusted_rand_score(true_labels, labels_aic)

In [29]:
print(labels_agg)
print(labels_kmeans)
print(labels_ms)
print(labels_bic)
print(labels_aic)

num0 = 0
num1 = 0
num2 = 0
num3 = 0
num4 = 0
num5 = 0
for i in labels_aic:
    if i == 0:
        num0 += 1
    if i == 1:
        num1 += 1
    if i == 2:
        num2 += 1
    if i == 3:
        num3 += 1
    if i == 4:
        num4 += 1
    if i == 5:
        num5 += 1

print(num0)
print(num1)
print(num2)
print(num3)
print(num4)
print(num5)

[0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0]
[0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0]
[0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 2 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 3 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0
 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[3 1 4 4 3 4 1 2 4 1 1 4 4 1 4 2 4 4 1 0 1 0 3 0 3 4 1 4 2 4 4 1 0 3 0 3 5
 2 1 3 4 4 3 4 4 1 4 4 4 3 4 4 0 0 4 4 3 3 4 4 4 3 2 4 3 4 1 2 2 4 4 4 1 0
 4 3 4 1 3 3 3 1 4 4 4 3 3 1 2 4 4]
8
16
8
19
39
1


In [26]:
print(ari_agg)
print(ari_kmeans)
print(ari_ms)
print(ari_bic)
print(ari_aic)

-0.0033041323391952695
-0.0030821954838096536
0.003178536405918525
0.0009484176648178812
-0.0026142735610628484
